In [1]:
import pandas as pd
df = pd.read_hdf('./data.h5')
print(df.shape)
print(df.head())
print(df.sample(3))
'''The store turnover'''
df_turnover = df['price_total'].sum()
print(df_turnover)
print("...............................")
'''Number of customers'''
df_customer = df[True == df.is_canceled]['price_total'].sum()
print(df_customer / 100)
print(df['customer_id'].nunique())
print(df['product_id'].nunique())
print(df['order_id'].nunique())
average_sale = df['price_total'].sum() / df['order_id'].nunique()
print(average_sale)

(820906, 9)
   order_id  customer_id  product_id  quantity  price_unit  price_total  \
0         0            0           0         6         254         1524   
1         0            0           1         6         339         2034   
2         0            0           2         8         275         2200   
3         0            0           3         6         339         2034   
4         0            0           3         6         339         2034   

   country_id          order_date  is_canceled  
0           0 2010-12-01 08:26:00        False  
1           0 2010-12-01 08:26:00        False  
2           0 2010-12-01 08:26:00        False  
3           0 2010-12-01 08:26:00        False  
4           0 2010-12-01 08:26:00        False  
        order_id  customer_id  product_id  quantity  price_unit  price_total  \
648972     42844         1714          74        10         195         1950   
428179     26772          305         511         3         295          885   
241

In [29]:
import pandas as pd
import helper as h
df = pd.read_hdf('./data.h5')
print(df.shape)
# print(df.sample(5))
# print('.......................')
# print('head', df.head)
# print('.......................')
# print('tail', df.tail)
# print(df[['order_id', 'product_id', 'customer_id']])
# print('....................................')
# print(df.isnull().sum())
# print('....................................')
t_price = (
    df[['order_id','price_total']]
    .groupby('order_id')
    .agg('sum')
    .sort_values(by='price_total', ascending=False)
    .reset_index()
    )
total = t_price['price_total'].sum()
print(total)


(820906, 9)
1816145630


In [44]:
import pandas as pd
import helper as h
df = pd.read_hdf('./data.h5')
sorted_order = (
    df[['customer_id', 'price_total']]
    .groupby('customer_id')
    .agg('sum')
    .sort_values(by='price_total', ascending=False)
    .reset_index()
    )
print(sorted_order['price_total'].max())
sorted_order['price_cumsum'] = sorted_order['price_total'].cumsum()
print(sorted_order.head(5))
max_val = sorted_order['price_cumsum'].tail(1).values[0]
value_80prc = max_val * 0.8
top_client = sorted_order[sorted_order['price_cumsum'] < value_80prc]['customer_id']
print('....................')
print(top_client)


61130939
   customer_id  price_total  price_cumsum
0          448     61130939      61130939
1          907     53007160     114138099
2         2647     33695110     147833209
3          179     31039769     178872978
4           66     29774259     208647237
....................
0        448
1        907
2       2647
3        179
4         66
        ... 
1319    2142
1320    1550
1321    1584
1322    1135
1323     302
Name: customer_id, Length: 1324, dtype: int64
